<a href="https://colab.research.google.com/github/jmcaussade/Analitica_textual_con_ML_PF/blob/main/Proyecto_Final_Dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [1]:
!pip install tqdm
!pip install datasets
!pip install langdetect
!pip install num2words
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: file

In [2]:
import pandas as pd
import os
import re
import string
import torch
from num2words import num2words
from tqdm import tqdm
from datasets import load_dataset
from langdetect import detect
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2TokenizerFast
from spellchecker import SpellChecker
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split



## Para trabajar con un pedazo del dataset orginal

In [3]:
## SAMPLE DEL DATASET
dataset = load_dataset("somosnlp/recetas-cocina")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
df = dataset['train'].to_pandas()

# Sample the first 5000 rows from the DataFrame
#df = df.head(5000)

# Show the first few rows of the sampled data
df.head()

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25


In [5]:
df

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25
...,...,...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,https://www.recetasgratis.net/receta-de-berenj...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...,b0a6454b-a023-44af-a189-92dee8948490
28234,Puré de patatas con queso y huevo,https://www.recetasgratis.net/receta-de-pure-d...,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p...",6e52da68-9496-47de-b85c-7be7fd0b6ac2
28235,Rollitos de calabacín rellenos de jamón y queso,https://www.recetasgratis.net/receta-de-rollit...,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...,22412360-37fe-47de-9401-d3f1e95fe4e7
28236,Calabacín relleno de quinoa,https://www.recetasgratis.net/receta-de-calaba...,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...,eedfabb6-d14f-46e4-a34f-dc07b9015636


#1) Pre-processing



## Limpieza dataset


In [6]:
df = df.drop(columns=["url", "uuid"])
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [7]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
title          5597
ingredients       1
steps             0
dtype: int64


In [8]:
# Drop rows where 'title', "ingredients" y "steps" is null
df = df.dropna(subset=['title'])
df = df.dropna(subset=['ingredients'])
df = df.dropna(subset=['steps'])



In [9]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3223


In [10]:
# Drop duplicate rows
df = df.drop_duplicates()

In [11]:
print(df.dtypes)

title          object
ingredients    object
steps          object
dtype: object


In [12]:
df.head(100)

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...


In [13]:
df.shape

(19417, 3)

## Cambiar números a texto

In [14]:
# Function to convert numbers to words in Spanish
def convert_number_to_spanish(text):
    if not isinstance(text, str):
        return text  # Return original value if not a string

    # Split string into list of words
    temp_str = text.split()
    # Initialize an empty list for processed words
    new_string = []

    for word in temp_str:
        # If word is a digit, convert the digit to words in Spanish
        if word.isdigit():
            temp = num2words(int(word), lang='es')
            new_string.append(temp)
        else:
            # Append the word as it is
            new_string.append(word)

    # Join the words in new_string to form a single string
    return ' '.join(new_string)

In [15]:
# Apply the function to the 'steps' column
df['converted_steps'] = df['steps'].apply(convert_number_to_spanish)
df['converted_ingredients'] = df['ingredients'].apply(convert_number_to_spanish)


In [16]:
df.head(100)

,title,ingredients,steps,converted_steps,converted_ingredients
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla ½ taza de ceboll...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne ¼ taza de aceite de o...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel ½ cucharadita d...


## Cambiar fracciones a texto

In [17]:
# Mapping of Unicode fractions to words
unicode_fraction_map = {
    '¼': 'un cuarto',
    '½': 'un medio',
    '¾': 'tres cuartos',
    '⅓': 'un tercio',
    '⅔': 'dos tercios',
    '⅕': 'un quinto',
    '⅖': 'dos quintos',
    '⅗': 'tres quintos',
    '⅘': 'cuatro quintos',
    '⅙': 'un sexto',
    '⅚': 'cinco sextos',
    '⅛': 'un octavo',
    '⅜': 'tres octavos',
    '⅝': 'cinco octavos',
    '⅞': 'siete octavos',
}

# Function to replace fractions in text
def convert_unicode_fractions(text):
    if not isinstance(text, str):
        return text  # Skip non-string inputs

    # Replace each Unicode fraction with its text equivalent
    for unicode_fraction, word in unicode_fraction_map.items():
        text = text.replace(unicode_fraction, word)

    return text


In [18]:
# Apply the function to the 'steps' column
df['converted_steps_2'] = df['converted_steps'].apply(convert_unicode_fractions)

df['converted_ingredients_2'] = df['converted_ingredients'].apply(convert_unicode_fractions)

In [19]:
df.head(100)

,title,ingredients,steps,converted_steps,converted_ingredients,converted_steps_2,converted_ingredients_2
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...","Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...","En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...","Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...","En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...,...,...,...,...
95,Arroz Atollado de Pescado,2 cucharadas de mantequilla\r\n½ taza de cebol...,Calentar la mantequilla en una olla mediana a ...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla ½ taza de ceboll...,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla un medio taza de...
96,Arepas de Plátano Maduro,2 plátanos maduros con cascara lavados y corta...,Cocine los plátanos con la cascara en agua dur...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,1 libra de pasta penne\r\n¼ taza de aceite de ...,Cocine la pasta en agua hirviendo con sal hast...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne ¼ taza de aceite de o...,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,2 pechugas de pollo sin piel\r\n½ cucharadita...,Caliente la parrilla a fuego medio. Coloque el...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel ½ cucharadita d...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...


In [20]:
df = df.drop(columns=["steps", "ingredients", "converted_steps", "converted_ingredients"])

In [21]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ..."
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...
...,...,...,...
95,Arroz Atollado de Pescado,Calentar la mantequilla en una olla mediana a ...,dos cucharadas de mantequilla un medio taza de...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...


## Cambiar todo a español

In [22]:
# Function to detect the language of a given text
def detect_language(text):
    try:
        return detect(text)  # Detects the language and returns a language code (e.g., 'es' for Spanish)
    except:
        return None  # Return None if language detection fails

In [23]:
# Apply the language detection function to the 'steps' column
df['language'] = df['converted_steps_2'].apply(detect_language)

# Check for rows where the language is not Spanish ('es')
non_spanish_rows = df[df['language'] != 'es']

# Display rows in non-Spanish languages
print(non_spanish_rows)



                                          title  \
23                Sancocho de Gallina o Valluno   
8218                 Ceviche de Camarones facil   
8233            Enchiladas Morelianas Mexicanas   
8301               verdaderas enchiladas suizas   
8314             Croquetas de bacalao y cebolla   
...                                         ...   
27274                           PASTEL DE ELOTE   
27625                       Caviar de Berenjena   
27633            Pimientos Rellenos de Quesitos   
27658  Tarta de puerros panceta y queso Gruyere   
27981                       Fresh broccoli soup   

                                       converted_steps_2  \
23     In a large pot, place the chicken, corn, aliño...   
8218   uno SOFRIE LOS CAMARONES EN UN SARTEN ,PICA LA...   
8233   uno PONES A COCER PECHUGAS CON UN PEDAZO DE CE...   
8301   uno SE LE QUITA LA SEMILLA AL CHILE Y LA COLA ...   
8314   uno EN UN CALDERO SOFREIR LA CEBOLLA CON LA MA...   
...                        

In [24]:
#Drop las rows que no estàn en español
df = df[df['language'] == 'es']

In [25]:
# Count the number of rows where the language is not 'es' (not Spanish)
non_spanish_count = (df['language'] != 'es').sum()

# Print the result
print(f"Number of rows with a language different from Spanish: {non_spanish_count}")


Number of rows with a language different from Spanish: 0


## Check misspelled words


In [26]:
#!pip install tqdm

In [27]:
# from tqdm.notebook import tqdm
# tqdm.pandas()

In [28]:
# # Initialize the spell checker for Spanish
# spell = SpellChecker(language='es')

# # Function to correct misspellings while handling capitalization
# def correct_misspellings_with_case(text):
#     if not isinstance(text, str):
#         return text

#     words = text.split()
#     corrected_words = []

#     for word in words:
#         # Preserve capitalization if the word is capitalized
#         is_capitalized = word[0].isupper()
#         corrected_word = spell.correction(word.lower())  # Lowercase for checking

#         # If correction is found, apply capitalization if needed
#         if corrected_word:
#             if is_capitalized:
#                 corrected_word = corrected_word.capitalize()  # Restore capitalization
#             corrected_words.append(corrected_word)
#         else:
#             corrected_words.append(word)  # Use original word if no correction

#     return ' '.join(corrected_words)



In [29]:
# !pip install openpyxl
# !pip install xlsxwriter

In [30]:
# # Apply the function to your dataset
# df['corrected_steps'] = df['converted_steps_2'].apply(correct_misspellings_with_case)
# df['corrected_ingredients'] = df['converted_ingredients_2'].apply(correct_misspellings_with_case)
# df['corrected_title'] = df['title'].apply(correct_misspellings_with_case)

# df.to_excel('ortografia_corregida.xlsx', index=False)

## To lower case

In [31]:
df['corrected_title'] = df['title'].str.lower()

df['corrected_ingredients'] = df['converted_ingredients_2'].str.lower()

df['corrected_steps'] = df['converted_steps_2'].str.lower()

<ipython-input-31-1056a9bab1aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corrected_title'] = df['title'].str.lower()
<ipython-input-31-1056a9bab1aa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corrected_ingredients'] = df['converted_ingredients_2'].str.lower()


## Deshacerse de caracteres especiales

In [32]:
# # Function to remove punctuation and special characters
# def remove_punctuation(text):
#     if not isinstance(text, str):
#         return text  # Return original value if not a string
#     return text.translate(str.maketrans('', '', string.punctuation))

In [33]:
# # Use .loc to apply the function to the 'converted_steps_2' column
# df.loc[:, 'corrected_title'] = df['corrected_title'].apply(remove_punctuation)
# df.loc[:, 'corrected_steps'] = df['corrected_steps'].apply(remove_punctuation)
# df.loc[:, 'corrected_ingredients'] = df['corrected_ingredients'].apply(remove_punctuation)


In [34]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e..."
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u..."


## Concatenar columnas

In [35]:
# Concatenate the columns and create a new column in the DataFrame
df['combined'] = "Titulo:" + " " + df['corrected_title'] + " " + "Ingredientes:"+ " " + df['corrected_ingredients'] + " " + "Receta:" + " " + df['corrected_steps']

In [36]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps,combined
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...
...,...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...


## Contar numero de palabras

In [37]:
# Iterate through each row in the DataFrame and add word counts to the new columns
df.loc[:, 'steps_word_count'] = df['corrected_steps'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df.loc[:, 'ingredients_word_count'] = df['corrected_ingredients'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df.loc[:, 'combined_word_count'] = df['combined'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)


# Now the DataFrame has new columns with the word counts for each row
# You can calculate the min, max, and average for these new columns

# For 'converted_steps_2'
steps_min = df['steps_word_count'].min()
steps_max = df['steps_word_count'].max()
steps_avg = df['steps_word_count'].mean()

# For 'ingredients'
ingredientes_min = df['ingredients_word_count'].min()
ingredientes_max = df['ingredients_word_count'].max()
ingredientes_avg = df['ingredients_word_count'].mean()

# For 'combined'
combined_min = df['combined_word_count'].min()
combined_max = df['combined_word_count'].max()
combined_avg = df['combined_word_count'].mean()

# Print the results
print(f"Converted Steps 2 - Min: {steps_min}, Max: {steps_max}, Average: {steps_avg:.2f}")
print(f"Converted Ingredientes 2- Min: {ingredientes_min}, Max: {ingredientes_max}, Average: {ingredientes_avg:.2f}")
print(f"Combined - Min: {combined_min}, Max: {combined_max}, Average: {combined_avg:.2f}")





Converted Steps 2 - Min: 6, Max: 1813, Average: 201.17
Converted Ingredientes 2- Min: 2, Max: 209, Average: 40.98
Combined - Min: 22, Max: 1880, Average: 249.59


In [38]:
df.head(100)

,title,converted_steps_2,converted_ingredients_2,language,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count
0,Arepas de Queso,"Combine la harina de maíz, agua caliente, el q...",uno taza de harina de arepa blanca o amarilla ...,es,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127
1,Sudado de Pollo,"En una olla grande, caliente el aceite vegetal...",ocho muslos de pollo sin la piel uno cucharada...,es,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182
2,Sancocho Trifásico,"Coloque la cebolla, el pimientón, el ajo y el ...",uno taza de cebolla picada uno pimientón rojo ...,es,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222
3,Sudado de Posta o Muchacho,"En una olla grande ponga la carne, el caldo, e...","uno posta o muchacho de tres a cuatro libras, ...",es,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165
4,Sopa de Lentejas Colombiana,Cocinar el chorizo en el aceite en una olla gr...,uno cucharadita de aceite vegetal uno taza de ...,es,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210
...,...,...,...,...,...,...,...,...,...,...,...
96,Arepas de Plátano Maduro,Cocine los plátanos con la cascara en agua dur...,dos plátanos maduros con cascara lavados y cor...,es,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...,96,55,158
97,Pasta con Salsa de Champiñones,Cocine la pasta en agua hirviendo con sal hast...,uno libra de pasta penne un cuarto taza de ace...,es,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...,147,65,220
98,Ensalada de Pollo a la Plancha con Aderezo de ...,Caliente la parrilla a fuego medio. Coloque el...,dos pechugas de pollo sin piel un medio cuchar...,es,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...,129,100,244
99,Tortilla Paisa o Antioqueña,"En una sartén antiadherente grande, caliente u...",tres cucharadas de aceite vegetal cuatro huevo...,es,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...,133,31,171


In [39]:
df = df.drop(columns=["title", "converted_steps_2", "converted_ingredients_2", "language"])

In [40]:
# Count rows where 'steps_word_count' is greater than 400
count = df[df['combined_word_count'] < 300].shape[0]

# Print the result
print(f"Number of rows with 'steps_word_count' greater than 400: {count}")


Number of rows with 'steps_word_count' greater than 400: 13518


#2) Tokenization

In [42]:
# Load the Spanish GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("datificate/gpt2-small-spanish")

# Enable tqdm for pandas
tqdm.pandas(desc="Tokenizing data")

# Check if the tokenizer is loaded properly
print(tokenizer)


GPT2TokenizerFast(name_or_path='datificate/gpt2-small-spanish', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [43]:
df['combined_tokens'] = df['combined'].progress_apply(lambda x: tokenizer.encode(x, add_special_tokens=True))


Tokenizing data: 100%|██████████| 19272/19272 [00:20<00:00, 935.32it/s]


In [44]:
# Add a column to store the length of tokenized sequences
df['combined_tokens_length'] = df['combined_tokens'].apply(len)

# Generate summary statistics for token lengths
print("Summary statistics for 'combined_tokens_length':")
print(df['combined_tokens_length'].describe())

# Display the distribution of token lengths
print("\nDistribution of token lengths:")
print(df['combined_tokens_length'].value_counts().sort_index())

# Identify rows with the longest and shortest tokenized sequences
max_length = df['combined_tokens_length'].max()
min_length = df['combined_tokens_length'].min()

longest_rows = df[df['combined_tokens_length'] == max_length]
shortest_rows = df[df['combined_tokens_length'] == min_length]

print("\nRows with the longest tokenized sequences:")
print(longest_rows[['combined', 'combined_tokens']])

print("\nRows with the shortest tokenized sequences:")
print(shortest_rows[['combined', 'combined_tokens']])


Summary statistics for 'combined_tokens_length':
count    19272.000000
mean       346.269614
std        167.734373
min         41.000000
25%        225.000000
50%        325.000000
75%        439.000000
max       2503.000000
Name: combined_tokens_length, dtype: float64

Distribution of token lengths:
combined_tokens_length
41      1
42      1
46      2
47      2
49      2
       ..
1694    1
1785    1
2311    1
2362    1
2503    1
Name: count, Length: 933, dtype: int64

Rows with the longest tokenized sequences:
                                                combined  \
21492  Titulo: ¡un postre delicioso! Ingredientes: tr...   

                                         combined_tokens  
21492  [52, 1596, 362, 26, 12503, 282, 21796, 309, 36...  

Rows with the shortest tokenized sequences:
                                                combined  \
15851  Titulo: ensalada tortrix Ingredientes: lechuga...   

                                         combined_tokens  
15851  [52, 1596, 

In [45]:
df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
0,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127,"[52, 1596, 362, 26, 22297, 2075, 258, 15968, 2...",174
1,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182,"[52, 1596, 362, 26, 308, 2487, 258, 24007, 276...",260
2,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222,"[52, 1596, 362, 26, 2135, 400, 750, 34522, 803...",297
3,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165,"[52, 1596, 362, 26, 308, 2487, 258, 621, 291, ...",225
4,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210,"[52, 1596, 362, 26, 29650, 258, 24714, 1552, 1...",291
...,...,...,...,...,...,...,...,...,...
96,arepas de plátano maduro,dos plátanos maduros con cascara lavados y cor...,cocine los plátanos con la cascara en agua dur...,Titulo: arepas de plátano maduro Ingredientes:...,96,55,158,"[52, 1596, 362, 26, 22297, 2075, 258, 29463, 2...",221
97,pasta con salsa de champiñones,uno libra de pasta penne un cuarto taza de ace...,cocine la pasta en agua hirviendo con sal hast...,Titulo: pasta con salsa de champiñones Ingredi...,147,65,220,"[52, 1596, 362, 26, 22059, 295, 18572, 258, 22...",322
98,ensalada de pollo a la plancha con aderezo de ...,dos pechugas de pollo sin piel un medio cuchar...,caliente la parrilla a fuego medio. coloque el...,Titulo: ensalada de pollo a la plancha con ade...,129,100,244,"[52, 1596, 362, 26, 27261, 366, 258, 24007, 26...",346
99,tortilla paisa o antioqueña,tres cucharadas de aceite vegetal cuatro huevo...,"en una sartén antiadherente grande, caliente u...",Titulo: tortilla paisa o antioqueña Ingredient...,133,31,171,"[52, 1596, 362, 26, 1115, 2946, 547, 3352, 330...",236


In [46]:
# Count rows where the length of 'combined_tokens' exceeds 512
rows_with_more_than_512_tokens = df[df['combined_tokens'].apply(len) > 512]

# Get the count of such rows
count = rows_with_more_than_512_tokens.shape[0]

print(f"Number of rows with more than 512 tokens: {count}")


Number of rows with more than 512 tokens: 2792


In [47]:
# Filter out rows with more than 512 tokens
df_filtered = df[df['combined_tokens'].apply(len) <= 512]

# Print the new size of the DataFrame
print(f"Original dataset size: {df.shape[0]}")
print(f"Filtered dataset size: {df_filtered.shape[0]}")


Original dataset size: 19272
Filtered dataset size: 16480


### Check For Null Values

In [48]:
# Check for null values in the 'combined_tokens' column
null_values_combined_tokens = df_filtered['combined_tokens'].isnull().sum()

# Print the result
print(f"Number of null values in 'combined_tokens': {null_values_combined_tokens}")


Number of null values in 'combined_tokens': 0


### Check for Nan values


In [49]:
# Check for NaN values in the 'combined_tokens' column
nan_values_combined_tokens = df_filtered['combined_tokens'].isna().sum()

# Print the result
print(f"Number of NaN values in 'combined_tokens': {nan_values_combined_tokens}")


Number of NaN values in 'combined_tokens': 0


In [50]:
# Display the first few rows of the 'combined_tokens' column
print(df_filtered['combined_tokens'].head())


0    [52, 1596, 362, 26, 22297, 2075, 258, 15968, 2...
1    [52, 1596, 362, 26, 308, 2487, 258, 24007, 276...
2    [52, 1596, 362, 26, 2135, 400, 750, 34522, 803...
3    [52, 1596, 362, 26, 308, 2487, 258, 621, 291, ...
4    [52, 1596, 362, 26, 29650, 258, 24714, 1552, 1...
Name: combined_tokens, dtype: object


### Check for empty tokens list

In [51]:
# Check for empty token lists in the 'combined_tokens' column
empty_tokens = df_filtered[df_filtered['combined_tokens'].apply(lambda x: len(x) == 0)]

# Print the number of rows with empty token lists
print(f"Number of rows with empty token lists: {len(empty_tokens)}")

# Optionally, display the first few rows with empty token lists
print(empty_tokens.head())


Number of rows with empty token lists: 0
Empty DataFrame
Columns: [corrected_title, corrected_ingredients, corrected_steps, combined, steps_word_count, ingredients_word_count, combined_word_count, combined_tokens, combined_tokens_length]
Index: []


In [52]:
# Get descriptive statistics for the 'combined_tokens' column
tokens_description = df_filtered['combined_tokens'].apply(len).describe()

# Print the descriptive statistics
print(tokens_description)


count    16480.000000
mean       296.521117
std        110.699017
min         41.000000
25%        211.000000
50%        295.000000
75%        382.000000
max        512.000000
Name: combined_tokens, dtype: float64


### Define padding token

In [53]:
# Manually set the padding token ID (using EOS token as pad token)
tokenizer.pad_token = tokenizer.eos_token  # Use end-of-sequence token as pad token
tokenizer.pad_token_id = tokenizer.encode(tokenizer.pad_token)[0]  # Get token ID for pad_token

# Check if the pad_token_id is set correctly
print(f"Pad Token ID: {tokenizer.pad_token_id}")

Pad Token ID: 0


### Padding to max lenght

In [54]:
# Padding to max length
max_length = 512  # Define your desired maximum length

# Apply padding to the tokenized data
# Correct way to set the values using .loc to avoid SettingWithCopyWarning
df_filtered.loc[:, 'combined_tokens'] = df_filtered['combined_tokens'].apply(
    lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length]
)


In [55]:
# Check the pad_token_id to ensure it's valid
print(f"Pad Token ID: {tokenizer.pad_token_id}")


Pad Token ID: 0


### Check for invalid tokens

In [56]:
# Check for invalid token IDs (e.g., negative or zero values)
invalid_directions = df[df['combined_tokens'].apply(lambda x: any(i < 0 for i in x))]

print(f"Invalid combined tokens: {len(invalid_directions)}")



Invalid combined tokens: 0


### Check all values are lists

In [57]:
# Check that all values are lists
non_list_combined = df[~df['combined_tokens'].apply(lambda x: isinstance(x, list))]

print(f"Non-list combined tokens: {len(non_list_combined)}")

Non-list combined tokens: 0


### Check for non integer values

In [58]:
# Check for non-integer values in the token lists
invalid_combined_values = df[df['combined_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]

print(f"Invalid combined token values: {len(invalid_combined_values)}")

Invalid combined token values: 0


### Check datatypes

In [59]:
# Check data types
print(df['combined_tokens'].apply(type).value_counts())

combined_tokens
<class 'list'>    19272
Name: count, dtype: int64


### Last inspection before Trainning

In [60]:
df["combined_tokens"].head(100)

,combined_tokens
0,"[52, 1596, 362, 26, 22297, 2075, 258, 15968, 2..."
1,"[52, 1596, 362, 26, 308, 2487, 258, 24007, 276..."
2,"[52, 1596, 362, 26, 2135, 400, 750, 34522, 803..."
3,"[52, 1596, 362, 26, 308, 2487, 258, 621, 291, ..."
4,"[52, 1596, 362, 26, 29650, 258, 24714, 1552, 1..."
...,...
96,"[52, 1596, 362, 26, 22297, 2075, 258, 29463, 2..."
97,"[52, 1596, 362, 26, 22059, 295, 18572, 258, 22..."
98,"[52, 1596, 362, 26, 27261, 366, 258, 24007, 26..."
99,"[52, 1596, 362, 26, 1115, 2946, 547, 3352, 330..."


In [62]:
df

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
0,arepas de queso,uno taza de harina de arepa blanca o amarilla ...,"combine la harina de maíz, agua caliente, el q...",Titulo: arepas de queso Ingredientes: uno taza...,92,29,127,"[52, 1596, 362, 26, 22297, 2075, 258, 15968, 2...",174
1,sudado de pollo,ocho muslos de pollo sin la piel uno cucharada...,"en una olla grande, caliente el aceite vegetal...",Titulo: sudado de pollo Ingredientes: ocho mus...,92,84,182,"[52, 1596, 362, 26, 308, 2487, 258, 24007, 276...",260
2,sancocho trifásico,uno taza de cebolla picada uno pimientón rojo ...,"coloque la cebolla, el pimientón, el ajo y el ...",Titulo: sancocho trifásico Ingredientes: uno t...,103,114,222,"[52, 1596, 362, 26, 2135, 400, 750, 34522, 803...",297
3,sudado de posta o muchacho,"uno posta o muchacho de tres a cuatro libras, ...","en una olla grande ponga la carne, el caldo, e...",Titulo: sudado de posta o muchacho Ingrediente...,85,72,165,"[52, 1596, 362, 26, 308, 2487, 258, 621, 291, ...",225
4,sopa de lentejas colombiana,uno cucharadita de aceite vegetal uno taza de ...,cocinar el chorizo en el aceite en una olla gr...,Titulo: sopa de lentejas colombiana Ingredient...,129,74,210,"[52, 1596, 362, 26, 29650, 258, 24714, 1552, 1...",291
...,...,...,...,...,...,...,...,...,...
28233,berenjenas rellenas de carne picada y bechamel...,cuatro berenjenas dos zanahorias uno cebolla u...,uno empezaremos a preparar estas berenjenas re...,Titulo: berenjenas rellenas de carne picada y ...,388,49,449,"[52, 1596, 362, 26, 13187, 267, 74, 2110, 1572...",649
28234,puré de patatas con queso y huevo,dos patatas dos huevos cien gramos de parmesan...,"uno cocemos las patatas y, una vez listas, las...",Titulo: puré de patatas con queso y huevo Ingr...,226,21,257,"[52, 1596, 362, 26, 15394, 313, 258, 29725, 29...",331
28235,rollitos de calabacín rellenos de jamón y queso,un medio calabacín cien gramos de jamón cocido...,uno lo primero que debemos hacer es lavar bien...,Titulo: rollitos de calabacín rellenos de jamó...,231,15,257,"[52, 1596, 362, 26, 23801, 1609, 258, 882, 337...",362
28236,calabacín relleno de quinoa,cuatro calabacines uno cebolla uno pimiento ve...,uno lo primero que tenemos que hacer para elab...,Titulo: calabacín relleno de quinoa Ingredient...,371,47,425,"[52, 1596, 362, 26, 882, 337, 467, 560, 23056,...",612


In [63]:
# Split into training and validation (80% train, 20% validation, for example)
train_df, valid_df = train_test_split(df_filtered, test_size=0.2, random_state=42)

# Now both train_df and valid_df are ready with tokenized, padded, and filtered data
print(f"Training set size: {train_df.shape[0]}")
print(f"Validation set size: {valid_df.shape[0]}")


Training set size: 13184
Validation set size: 3296


In [64]:
train_df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
14435,costillas de cerdo fritas con miel,dos costillar de cerdo uno vaso de vino blanco...,uno para preparar estas costillas de cerdo fri...,Titulo: costillas de cerdo fritas con miel Ing...,280,29,318,"[52, 1596, 362, 26, 30741, 258, 17677, 14304, ...",419
17743,pepinos en salmuera,dos kg. de pepinos 1/2 kg. de sal cuatro litro...,"uno se limpia los pepinos con un trapo húmedo,...",Titulo: pepinos en salmuera Ingredientes: dos ...,133,12,151,"[52, 1596, 362, 26, 821, 41831, 278, 798, 4645...",211
27778,tomate y chiles negros,diez tomatillos verdes siete chiles negros pas...,uno muchos la conocen como pico de gallo. se p...,Titulo: tomate y chiles negros Ingredientes: d...,121,13,141,"[52, 1596, 362, 26, 27255, 287, 1058, 2448, 87...",195
19921,pasta frita facilísima,doscientos gramos de espaguetis uno chorrito d...,uno cocer la pasta al dente.escurrir y reserva...,Titulo: pasta frita facilísima Ingredientes: d...,60,40,106,"[52, 1596, 362, 26, 22059, 14304, 291, 4627, 9...",172
13570,roast beef a la soja,dos cucharadas de aceite de oliva 1/2 onza de ...,uno calentar el aceite y la mantequilla en una...,Titulo: roast beef a la soja Ingredientes: dos...,117,38,163,"[52, 1596, 362, 26, 874, 6722, 5476, 1540, 265...",239
...,...,...,...,...,...,...,...,...,...
18264,jaiba michoacana,dos chiles uno aguacates dos pepino 1/2 ceboll...,uno primero pica todos los ingredientes por ej...,Titulo: jaiba michoacana Ingredientes: dos chi...,67,24,96,"[52, 1596, 362, 26, 4866, 12367, 285, 13923, 4...",142
12602,matambre a la pizza parrilla,uno kilogramo de matambre de cerdo veinticinco...,uno para comenzar con la preparación del matam...,Titulo: matambre a la pizza parrilla Ingredien...,246,36,290,"[52, 1596, 362, 26, 637, 493, 401, 265, 276, 2...",397
538,vegetales gratinados,1/4 libra de habichuelas cortadas en tiras (12...,1. precalienta el horno a 170°c (350°f). diez ...,Titulo: vegetales gratinados Ingredientes: 1/4...,134,77,216,"[52, 1596, 362, 26, 12753, 968, 488, 413, 2760...",340
15544,ensalada de pasta con guisantes,doscientos gramos de pasta penne o cualquier p...,uno antes de realizar la ensalada con guisante...,Titulo: ensalada de pasta con guisantes Ingred...,292,54,354,"[52, 1596, 362, 26, 27261, 366, 258, 22059, 29...",479


In [65]:
valid_df.head(100)

,corrected_title,corrected_ingredients,corrected_steps,combined,steps_word_count,ingredients_word_count,combined_word_count,combined_tokens,combined_tokens_length
20534,brochetas de rape y langostinos con salsa amer...,dieciséis medallones de rape dieciséis langost...,uno la salsa americana es lo primero que debem...,Titulo: brochetas de rape y langostinos con sa...,238,61,310,"[52, 1596, 362, 26, 10126, 1342, 442, 258, 969...",443
26260,sopa de patatas bien espesa,trescientos g de patatas curry una pizca de sa...,uno ponemos en la olla a presión el agua justa...,Titulo: sopa de patatas bien espesa Ingredient...,82,24,114,"[52, 1596, 362, 26, 29650, 258, 29725, 1329, 4...",164
22051,ganache de chocolate blanco para cubrir y rell...,cuatrocientos gramos de chocolate blanco dosci...,uno para poder realizar correctamente el relle...,Titulo: ganache de chocolate blanco para cubri...,286,14,312,"[52, 1596, 362, 26, 4958, 1342, 258, 21167, 38...",426
17494,habichuelas con patitas,2lb de patitas de cerdo uno pote de dieciséis ...,uno primero tiene que hervir 2lb de patitas de...,Titulo: habichuelas con patitas Ingredientes: ...,118,45,169,"[52, 1596, 362, 26, 498, 958, 3783, 295, 547, ...",256
13331,churrascos a la criolla,setecientos cincuenta gramos de churrasco uno ...,"uno preparar de antemano los churrascos, sazon...",Titulo: churrascos a la criolla Ingredientes: ...,186,44,237,"[52, 1596, 362, 26, 47955, 545, 857, 265, 276,...",304
...,...,...,...,...,...,...,...,...,...
19314,fideos con atún,dos latas de atún uno unidad de cebolla picada...,uno para empezar con la preparación de los fid...,Titulo: fideos con atún Ingredientes: dos lata...,227,55,288,"[52, 1596, 362, 26, 26131, 261, 295, 8106, 563...",418
17672,cangrejos a la soriana con tomates,uno kilo de cangrejos de río cien gramos de ce...,"uno en abundante agua hirviendo con sal, cocem...",Titulo: cangrejos a la soriana con tomates Ing...,111,30,150,"[52, 1596, 362, 26, 43540, 265, 276, 3535, 215...",211
24089,pudin de fresas y nata,quinientos gramos de fresa tres cucharadas sop...,uno haz un puré triturando las fresas ya limpi...,Titulo: pudin de fresas y nata Ingredientes: q...,110,50,168,"[52, 1596, 362, 26, 594, 4635, 258, 6754, 271,...",242
8359,canapes faciles,galletas abaneras queso crema pimienta sal alg...,"uno en un recipiente pon el queso crema, la pi...",Titulo: canapes faciles Ingredientes: galletas...,29,9,43,"[52, 1596, 362, 26, 262, 581, 4990, 1202, 9061...",68


#3) Trainning Phase

In [67]:
# Define a simple Dataset class to handle tokenized inputs
class RecipeDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.input_ids = df['combined_tokens'].tolist()
        self.attention_mask = [[1] * len(ids) for ids in self.input_ids]  # Attention mask is 1 for all tokens

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long)
        }

In [68]:
# Prepare the DataLoaders
train_dataset = RecipeDataset(train_df)
valid_dataset = RecipeDataset(valid_df)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=8)

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

scaler = GradScaler()

<ipython-input-68-0d89fc5e8e20>:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [69]:
print(torch.version.cuda)

12.1


In [70]:
# Check which device the model is loaded on
print(f"Model is loaded onto: {model.device}")


Model is loaded onto: cuda:0


In [71]:
# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

## Connect to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [73]:
# Path to the directory where you want to save the model
directory_path = "/content/drive/My Drive/2024-02/Analitica_textual_con_ML/Proyecto_Final/Model"

# Check if the directory exists
if not os.path.exists(directory_path):
    print(f"Directory '{directory_path}' does not exist. Creating it now.")
    os.makedirs(directory_path)  # Create the directory if it doesn't exist
else:
    print(f"Directory '{directory_path}' already exists.")


Directory '/content/drive/My Drive/2024-02/Analitica_textual_con_ML/Proyecto_Final/Model' already exists.


In [74]:
# Save model and optimizer
model_save_path = "/content/drive/My Drive/2024-02/Analitica_textual_con_ML/Proyecto_Final/Model/gpt2_spanish_model_epoch_{}.pt"
optimizer_save_path = "/content/drive/My Drive/2024-02/Analitica_textual_con_ML/Proyecto_Final/Model/optimizer_epoch_{}.pt"

# Initialize GradScaler for mixed precision training
scaler = GradScaler()

epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", ncols=100):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        # Mixed precision forward pass using autocast
        with autocast():  # Enable mixed precision
            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()

        # Update model weights with scaled gradients
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_train_loss}")

    # Validation step with mixed precision
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f"Validating Epoch {epoch+1}", ncols=100):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Mixed precision forward pass using autocast
            with autocast():
                outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
                val_loss = outputs.loss
                total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(valid_dataloader)
    print(f"Epoch {epoch+1} - Average Validation Loss: {avg_val_loss}")

    # Save model and optimizer after each epoch
    torch.save(model.state_dict(), model_save_path.format(epoch+1))
    torch.save(optimizer.state_dict(), optimizer_save_path.format(epoch+1))


<ipython-input-74-9f0646d03628>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Training Epoch 1:   0%|                                                    | 0/1648 [00:00<?, ?it/s]<ipython-input-74-9f0646d03628>:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision
Training Epoch 1: 100%|█████████████████████████████████████████| 1648/1648 [11:44<00:00,  2.34it/s]


Epoch 1 - Average Training Loss: 1.5834361495251217


Validating Epoch 1:   0%|                                                   | 0/412 [00:00<?, ?it/s]<ipython-input-74-9f0646d03628>:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validating Epoch 1: 100%|█████████████████████████████████████████| 412/412 [00:58<00:00,  7.06it/s]


Epoch 1 - Average Validation Loss: 1.2990939890586057


Training Epoch 2: 100%|█████████████████████████████████████████| 1648/1648 [11:44<00:00,  2.34it/s]


...

Epoch 2 - Average Validation Loss: 1.2139755938816996


Training Epoch 3: 100%|█████████████████████████████████████████| 1648/1648 [11:44<00:00,  2.34it/s]


Epoch 3 - Average Training Loss: 1.201983830774004


Validating Epoch 3: 100%|█████████████████████████████████████████| 412/412 [00:58<00:00,  7.06it/s]


Epoch 3 - Average Validation Loss: 1.171319928273414


#4) Load Model

In [6]:
# Specify the path to your model file in Google Drive
model_path = '/content/drive/My Drive/2024-02/Analitica_textual_con_ML/Proyecto_Final/Model/gpt2_spanish_model_epoch_3.pt'

# Load the model (ensure that the same model configuration is used)
model = GPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")
model.load_state_dict(torch.load(model_path))

# Load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("datificate/gpt2-small-spanish")

# Ensure the model is moved to the same device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

<ipython-input-6-dc8bb2cde8eb>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
print(model.device)  # Should print 'cuda:0' if it's on GPU


cuda:0


#5) Assistant

In [8]:
print(next(model.parameters()).device)


cuda:0


In [12]:
def generate_response(prompt, model, tokenizer, max_length=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length,
        temperature=0.8,  # Add randomness
        top_k=50,        # Limit to top-k most likely tokens
        top_p=0.9,       # Nucleus sampling
        repetition_penalty=1.2,  # Penalize repetition
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


## Prompt


In [14]:
# Example prompt for the cooking assistant
prompt = "Cómo hacer panqueques?"

# Use the generate_response function to get a model response
response = generate_response(prompt, model, tokenizer, max_length=150)

# Print the generated response
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Cómo hacer panqueques? Ingredientes: uno taza de harina cero (140 gramos) dos cucharadas soperas de aceite vegetal o girasol un medio taza de agua tibia tres tazas de leche entera Receta: uno para empezar con la receta de pancakes caseros, primero debes mezclar el aceite y la leche. luego, agrega los ingredientes secos en forma envolvente hasta que se integren bien todos estos ingredientes. truco: si no tienes una licuadora puedes usar una batidora eléctrica sin problemas ni añadir nada más al polvo del pancocho casero porque es muy fácil triturar las harinas. dos a continuación, añade poco a poco el agua caliente junto con la harina tamizada e integra todo perfectamente.
